import import import

In [0]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

import warnings

In [0]:
df_jul2019 = pd.read_excel (r'/content/drive/My Drive/salesByMonth/jul2019.xls')
df_apr2020 = pd.read_excel (r'/content/drive/My Drive/salesByMonth/apr2020.xls')
df_jan2020 = pd.read_excel (r'/content/drive/My Drive/salesByMonth/jan2020.xls')
df_aug2019 = pd.read_excel (r'/content/drive/My Drive/salesByMonth/aug2019.xls')
df_sep2019 = pd.read_excel (r'/content/drive/My Drive/salesByMonth/sep2019.xls')
df_oct2019 = pd.read_excel (r'/content/drive/My Drive/salesByMonth/oct2019.xls')
df_nov2019 = pd.read_excel (r'/content/drive/My Drive/salesByMonth/nov2019.xls')
df_dec2019 = pd.read_excel (r'/content/drive/My Drive/salesByMonth/dec2019.xls')
df_feb2020 = pd.read_excel (r'/content/drive/My Drive/salesByMonth/feb2020.xls')
df_mar2020 = pd.read_excel (r'/content/drive/My Drive/salesByMonth/mar2020.xls')

In [0]:
#month = ['jul2019','apr2020','jan2020','aug2019','sep2019','oct2019','nov2020','dec2019','feb2019','mar2019']
#for s in month:
#  df_

In [75]:
df_mar2020.head()

,序号,公司编号,客户新代码,客户名称,所属国家,收货地址,收货国家,运营模式,订单号,发货仓库,出库单号,发货时间,商品ID,存货ID,虚拟ID,电源线,los,供应商,CG号,中文名称,货物名称,规格型号,产品重量,发货总重量,销售地区,业务员,到款方式,到款币种,到款金额,打款时间,发票号,HF单号,IN编码,发票重开,销售币种,数量,订单单价,前台售价,折算汇率,含税单价,含税金额,其中销售税,客户GST税号,GST税率,GST金额,GST分摊,不含税金额（原币）,不含税金额（AUD）,个人/公司,运输方式,运单号,转运方式,转运单号,是否免运费,运费金额,分摊运费,保险费用,分摊保费,实际运费,采购单价,采购运费,采购成本原币,采购成本（AUD）,仓管ID,折人民币,国内采购成本,采购员
0,1,G851170324,D121226246,Ensbey Enterprises Pty Ltd,Australia,Niel Ensbey\n1 Treviso Court\n Urangan -- 4655...,Australia(内销),HW9,CW202002280538-AU1,澳洲仓,FSCK20022828034-AU,2020-03-02 09:15:36,80365,80365,0,NaN,NaN,750X,CG1906101021-AU,以太网交换机,S3260-8T2FP 8-Port Gigabit PoE+ Managed Switch...,S3260-8T2FP,1.800,2.4kg,非欧盟,Bessy.Yang,Paypal,AUD,345.40,2020-02-28,FS200227518068,NaN,IN052002280447,否,AUD,1,277.0,190.2734,1.4558,345.40,345.40,NaN,NaN,10%,31.40,31.40,314.0,314.0,个人,DHL Express 1-3 Business Days,5528728601,NaN,NaN,否,37.0,37.0,0.0,0.0,0.0,870.00,0,870.0,180.86,4284,"1,342.38",870.0,Alice
1,2,G951090651,D261226888,Territory Technology Solutions Pty Ltd,Australia,Purchasing Officer\n8 Albatross Street\n Winne...,Australia(内销),HW9,CW202002280553-AU1,澳洲仓,FSCK20022828038-AU,2020-03-02 09:16:03,74096,74096,0,NaN,NaN,241X,CG1909050066-AU,网线,4ft (1.2m) Cat6 Snagless Unshielded (UTP) PVC ...,无型,0.060,1.296kg,非欧盟,Annabella.Shen,Australian credit card,AUD,57.20,2020-02-28,FS200227358077,NaN,IN052002280450,否,AUD,10,3.7,2.5416,1.4558,5.72,57.20,NaN,89086570541,10%,5.20,5.20,52.0,52.0,公司,StarTrack Fixed Price Premium,J8PZ50007243,NaN,NaN,否,15.0,15.0,0.0,0.0,0.0,8.16,0,81.6,16.96,4284,179.31,81.6,Sydnie.Wang
2,3,G621013274,C541172582,ABB Australia LTD PTY,Australia,Harry Eigelaar\n1 Bapaume Rd\n Moorebank -- 21...,Australia(内销),HW9,CW202002280555-AU1,澳洲仓,FSCK20022828039-AU,2020-03-02 09:18:55,41759,41759,0,NaN,NaN,701X,CG1907095931-AU,光纤跳线,5m (16ft) LC UPC to SC UPC Duplex OM3 Multimod...,无型,0.100,0.1848kg,非欧盟,Harper.Wang,Australian credit card,AUD,35.64,2020-02-28,FS200227688082,NaN,IN052002280453,否,AUD,2,8.7,5.9761,1.4558,17.82,35.64,NaN,NaN,10%,3.24,3.24,32.4,32.4,个人,StarTrack Fixed Price Premium,J8PZ50007244,NaN,NaN,否,15.0,15.0,0.0,0.0,0.0,15.35,0,30.7,6.38,4284,84.32,30.7,Iris.Liu
3,4,G631169366,611192334,EQUINIX,Australia,"Rulino Alday\n200 Bourke Road, Alexandria\n Sy...",Australia(内销),HW9,CW202002280550-AU,澳洲仓,FSCK20022828037-AU,2020-03-02 09:20:46,82730,82730,0,NaN,NaN,916X,CG1910220261-AU,光纤适配器,2.5mm to 1.25mm Fiber Optical Adapter for Visu...,无型,0.015,0.14784kg,非欧盟,River.Zhou,delivery after payment 真实到款,AUD,150.00,2020-02-28,FS202002180045,NaN,IN052002280451,否,AUD,10,15.0,10.0000,1.4558,16.50,165.00,NaN,NaN,10%,15.00,15.00,150.0,150.0,个人,StarTrack Fixed Price Premium,J8PZ50007245,NaN,NaN,是,0.0,0.0,0.0,0.0,0.0,16.50,0,165.0,34.30,4284,629.01,165.0,Sydnie.Wang
4,5,G441171133,D111226875,QAIHC,Australia,Graham Kissell\n1/36 Russell Street\n South Br...,Australia(内销),HW9,CW202002280547-AU1,澳洲仓,FSCK20022828035-AU,2020-03-02 09:25:16,66616,66616,0,NaN,NaN,FS019X,CG1912095757-AU,光纤模块,Dell GP-10GSFP-T Compatible 10GBASE-T SFP+ Cop...,SFP-10G-T,0.045,5.25kg,非欧盟,Ivy.Qin,Australian credit card,AUD,1093.40,2020-02-28,FS200227408074,NaN,IN052002280452,否,AUD,4,86.0,59.0740,1.4558,95.65,382.58,NaN,97111116762,10%,99.40,34.78,347.8,347.8,公司,TNT 9:00 Express,158696070,NaN,NaN,否,72.0,3.8,0.0,0.0,0.0,228.00,0,912.0,189.59,4284,"1,667.07",912.0,lisa


In [76]:
df_mar2020.columns

Index(['序号', '公司编号', '客户新代码', '客户名称', '所属国家', '收货地址', '收货国家', '运营模式', '订单号',
       '发货仓库', '出库单号', '发货时间', '商品ID', '存货ID', '虚拟ID', '电源线', 'los', '供应商',
       'CG号', '中文名称', '货物名称', '规格型号', '产品重量', '发货总重量', '销售地区', '业务员', '到款方式',
       '到款币种', '到款金额', '打款时间', '发票号', 'HF单号', 'IN编码', '发票重开', '销售币种', '数量',
       '订单单价', '前台售价', '折算汇率', '含税单价', '含税金额', '其中销售税', '客户GST税号', 'GST税率',
       'GST金额', 'GST分摊', '不含税金额（原币）', '不含税金额（AUD）', '个人/公司', '运输方式', '运单号',
       '转运方式', '转运单号', '是否免运费', '运费金额', '分摊运费', '保险费用', '分摊保费', '实际运费', '采购单价',
       '采购运费', '采购成本原币', '采购成本（AUD）', '仓管ID', '折人民币', '国内采购成本', '采购员'],
      dtype='object')

In [0]:
df_all = pd.concat([df_jul2019,df_apr2020,df_jan2020,df_aug2019,df_sep2019,df_oct2019,df_nov2019,df_dec2019,df_feb2020,df_mar2020], axis=0, ignore_index=True)

In [78]:
df_all.sample()

,序号,公司编号,客户新代码,客户名称,所属国家,收货地址,收货国家,运营模式,订单号,发货仓库,出库单号,发货时间,商品ID,存货ID,虚拟ID,中文名称,货物名称,规格型号,产品重量,发货总重量,销售地区,业务员,到款方式,到款币种,到款金额,打款时间,发票号,销售币种,数量,前台售价,折算汇率,含税单价,含税金额,其中销售税,客户GST税号,GST税率,GST金额,GST分摊,不含税金额（原币）,不含税金额（AUD）,个人/公司,运输方式,运单号,转运方式,转运单号,是否免运费,运费金额,分摊运费,保险费用,实际运费,采购单价,采购运费,采购成本原币,采购成本（AUD）,仓管ID,折人民币,国内采购成本,采购员,电源线,los,供应商,CG号,HF单号,IN编码,发票重开,订单单价,分摊保费,8月运费对账单,客户编码,含税金额.1
6579,1260,NaN,D521177528,Belmont High School,Australia,"Callum Finley\n17, Tree Street\n Waurn Ponds -...",Australia(内销),HW9,CW202001150348-AU1,澳洲仓,FSCK20011524988-AU,2020-01-15 14:32:23,70607,70607,0,网线,4ft (1.2m) Cat6 Snagless Unshielded (UTP) PVC ...,无型,0.06,2.7136kg,非欧盟,Jodie.Feng,Australian credit card,AUD,430.05,2020-01-15,FS200114717428,AUD,2,2.4042,1.4558,3.78,7.55,NaN,NaN,10%,39.1,0.69,6.86,6.86,个人,StarTrack Premium,J8PZ50006245,NaN,NaN,是,0.0,0.0,0.0,0.0,6.42,0.0,12.84,2.67,4253,33,12.84,Sydnie.Wang,NaN,NaN,128X,CG1809272638-DE,NaN,NaN,NaN,3.43,NaN,NaN,NaN,NaN


In [79]:
df_all['运输方式'].value_counts()

StarTrack Premium                             12820
TNT Road Express 1-4 Business Days             4432
StarTrack Fixed Price Premium                  2207
TNT Overnight Express                          2198
StarTrack Premium 1-3 Business Days            1468
DHL Express Worldwide Service                   608
Pickup at warehouse                             604
Australia Post Standard                         457
Australia Post Express                          373
DHL Express 1-3 Business Days                   345
FedEx Normal Service                            103
FedEx Express Service                            97
StarTrack 1-5 Business Days                      93
TNT 9:00 Express                                 75
DHL Express Worldwide 1-3 Business Days          18
StarTrack Premium 1-3 Business                   17
Australia Post Express 2-4 Business Days         16
TNT 9:00 Express Service                         10
Australia Post Standard 3-10 Business Days        6
TNT SameDay 

In [0]:
df_all2 = df_all.rename({'运输方式': 'DeliveryMethod','发货时间':'PickUpTime','订单号':'OrderNumber','不含税金额（AUD）':'PriceWithoutGST(AUD)','运单号':'TrackingNumber'}, axis=1)

In [81]:
df_all2['DeliveryMethod'].unique()

array(['StarTrack Premium 1-3 Business Days', 'TNT Overnight Express',
       'TNT Road Express 1-4 Business Days', 'Australia Post Standard',
       'Pickup at warehouse', 'DHL Express Worldwide Service',
       'StarTrack Premium', 'FedEx Express Service',
       'FedEx Normal Service', 'Australia Post Express',
       'DHL Express 1-3 Business Days', 'Fastway Courier',
       'StarTrack Premium 1-3 Business', 'TNT 9:00 Express Service',
       'StarTrack Fixed Price Premium',
       'Australia Post Express\xa02-4\xa0Business Days',
       'StarTrack 1-5 Business Days', 'TNT 9:00 Express',
       'Australia Post Standard\xa03-10\xa0Business Days',
       'TNT 12:00 Express',
       'DHL Express Worldwide\xa01-3\xa0Business Days', 'TNT Express PC',
       'DHL Express Worldwide', 'TNT SameDay', nan, 'DHL Express',
       'pick up', 'TNT 10:00 Express'], dtype=object)

In [0]:
startrack_tnt = ['StarTrack Premium','TNT Road Express 1-4 Business Days','StarTrack Fixed Price Premium','TNT Overnight Express','StarTrack Premium 1-3 Business Days','StarTrack 1-5 Business Days','TNT 9:00 Express','StarTrack Premium 1-3 Business','TNT 9:00 Express Service','TNT SameDay','TNT 10:00 Express','TNT 12:00 Express','TNT Express PC']
notstartrack_tnt = ['Australia Post Standard',
       'Pickup at warehouse', 'DHL Express Worldwide Service',
       'FedEx Express Service',
       'FedEx Normal Service', 'Australia Post Express',
       'DHL Express 1-3 Business Days', 'Fastway Courier',
       'Australia Post Express 2-4 Business Days',
       'Australia Post Standard 3-10 Business Days',
       'DHL Express Worldwide 1-3 Business Days', 
       'DHL Express Worldwide', 'DHL Express','Australia Post Standard\xa03-10\xa0Business Days','Australia Post Express\xa02-4\xa0Business Days','DHL Express Worldwide\xa01-3\xa0Business Days',
       'pick up']
#startrack_tnt
for i in notstartrack_tnt:
  df_all2 = df_all2[df_all2['DeliveryMethod'] != i]



In [83]:
#df = df.drop(df[df.score < 50].index)
df_all2['DeliveryMethod'].value_counts()

StarTrack Premium                      12820
TNT Road Express 1-4 Business Days      4432
StarTrack Fixed Price Premium           2207
TNT Overnight Express                   2198
StarTrack Premium 1-3 Business Days     1468
StarTrack 1-5 Business Days               93
TNT 9:00 Express                          75
StarTrack Premium 1-3 Business            17
TNT 9:00 Express Service                  10
TNT SameDay                                4
TNT 12:00 Express                          2
TNT 10:00 Express                          2
TNT Express PC                             1
Name: DeliveryMethod, dtype: int64

In [84]:
#df_all2['DeliveryMethod'].isnull.sum()
#df_all2[df_all2['DeliveryMethod'].isnull().any(axis=1)]


tempdf = df_all2[df_all2['DeliveryMethod'].isnull()]
tempdf

,序号,公司编号,客户新代码,客户名称,所属国家,收货地址,收货国家,运营模式,OrderNumber,发货仓库,出库单号,PickUpTime,商品ID,存货ID,虚拟ID,中文名称,货物名称,规格型号,产品重量,发货总重量,销售地区,业务员,到款方式,到款币种,到款金额,打款时间,发票号,销售币种,数量,前台售价,折算汇率,含税单价,含税金额,其中销售税,客户GST税号,GST税率,GST金额,GST分摊,不含税金额（原币）,PriceWithoutGST(AUD),个人/公司,DeliveryMethod,TrackingNumber,转运方式,转运单号,是否免运费,运费金额,分摊运费,保险费用,实际运费,采购单价,采购运费,采购成本原币,采购成本（AUD）,仓管ID,折人民币,国内采购成本,采购员,电源线,los,供应商,CG号,HF单号,IN编码,发票重开,订单单价,分摊保费,8月运费对账单,客户编码,含税金额.1
15697,1116,G781016126,531124787,IT Partners (Int\\\\\\\'l) Pty Ltd,Australia,Nenad Savic\n9 Tara Grove\n Carnegie -- 3163\n...,Australia(内销),HW9,CW20191009248-AU1-01H-AU,澳洲仓,FSCK19102820000-AU,2019-10-28 11:01:24,36431,36431,0,光纤模块,Intel E10GSFPSR Compatible 1000BASE-SX and 10G...,SFP-10GSR-85,0.02,0.2kg,非欧盟,Shako.Hu,"未付款,先备货",AUD,0.00,2019-10-09,FS201910210057,AUD,4,18.0000,1.4558,28.60,114.40,NaN,25088576283,10%,10.40,10.40,104.00,104.00,个人,NaN,J8PZ50003779,NaN,NaN,否,0.0,0.0,0.0,0.0,56.0000,0.0,224.0000,45.59,4062,511.03,224.0,lisa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.00,NaN,NaN,531124787.0,NaN
21380,867,G271003249,591134078,Trustpower Ltd,New Zealand,Jonathon Gumbley\n108 Durham St\n Tauranga -- ...,New Zealand(外销),HW9,ZQ20181127012-CN-01H-CN-03H-AU,深圳备货区,FSCK19121984438-WH,2019-12-24 11:42:50,48863,48863,0,光纤模块,Juniper Networks JNP-QSFP-100G-LR4 Compatible ...,QSFP28-LR4-100G,0.11,0.6912kg,非欧盟,Jane.Liang,"未付款,先备货",NZD,0.00,2018-11-27,FS201811260031R1R3,NZD,1,799.0000,1.5618,1124.16,1124.16,NaN,NaN,0%,0.00,0.00,1124.16,1047.86,个人,NaN,777332136041,DHL 2-5 Business Days,3.50189e+09,否,0.0,0.0,0.0,0.0,989.2608,0.0,989.2608,922.12,4019,"5,040.73",2240.0,lisa,NaN,NaN,FS004C,PZQ20181127012-02,NaN,NaN,NaN,1124.16,NaN,NaN,NaN,NaN
21384,871,G271003249,591134078,Trustpower Limited,New Zealand,Brendan Steed\n108 Durham St\n -- 3110\n New ...,New Zealand(外销),HW9,CW20171117236-03H-AU,深圳备货区,FSCK19121984440-WH,2019-12-24 13:46:47,48863,48863,0,光纤模块,Juniper Networks JNP-QSFP-100G-LR4 Compatible ...,QSFP28-LR4-100G,0.11,0.6912kg,非欧盟,Jane.Liang,"未付款,先备货",NZD,0.00,2017-11-17,FS20171110450V2R3-P2,NZD,5,799.0000,1.5618,3063.36,15316.80,NaN,NaN,0%,0.00,0.00,15316.80,14277.24,个人,NaN,777332364827,DHL 2-5 Business Days,3.50189e+09,否,0.0,0.0,0.0,0.0,2695.7568,0.0,13478.7840,12563.97,4019,"68,680.42",11200.0,lisa,NaN,NaN,FS004C,PCW20171117236-01,NaN,NaN,NaN,3063.36,NaN,NaN,NaN,NaN
21497,94,NaN,D431217479,CPP,Australia,Leon Gilet\n2/500 Princes Hwy\n St Peters -- 2...,Australia(内销),HW9,CW201912180467-AU4,武汉成品仓,FSCK19122085282-WH,2019-12-27 11:05:39,41476,41476,0,网络模块,Cat6 RJ45 (8P8C) Unshielded Coupler Keystone I...,6IMOR-U,0.02,0.6912kg,非欧盟,Queenie.Yang,Australian credit card,AUD,22.90,2019-12-18,FS191217723844-P2,AUD,2,1.5112,1.4558,7.64,15.27,NaN,NaN,10%,1.30,0.87,14.40,14.40,个人,NaN,J8PZ50005806,DHL 2-5 Business Days,3.50264e+09,否,15.0,10.0,0.0,0.0,1.9360,0.0,3.8720,3.87,2571,21.17,7.4,Sydnie.Wang,NaN,NaN,716X,CG170523007491,NaN,NaN,NaN,2.20,NaN,NaN,NaN,NaN
21498,95,NaN,D431217479,CPP,Australia,Leon Gilet\n2/500 Princes Hwy\n St Peters -- 2...,Australia(内销),HW9,CW201912180467-AU4,武汉成品仓,FSCK19122085277-WH,2019-12-27 11:05:39,41438,41438,0,网络模块,Cat6 RJ45 (8P8C) Unshielded Coupler Keystone I...,6IMWH-U,0.02,0.6912kg,非欧盟,Queenie.Yang,Australian credit card,AUD,22.90,2019-12-18,FS191217723844-P2,AUD,1,1.5112,1.4558,7.63,7.63,NaN,NaN,10%,1.30,0.43,7.20,7.20,个人,NaN,J8PZ50005806,DHL 2-5 Business Days,3.50264e+09,否,15.0,5.0,0.0,0.0,1.9360,0.0,1.9360,1.94,2571,10.58,3.7,Sydnie.Wang,NaN,NaN,716X,CG1809016257-CN,NaN,NaN,NaN,2.20,NaN,NaN,NaN,NaN
21503,100,NaN,D201171303,Russell mineral Equipment,Australia,Nick Hurd\n313 Mcdougall street\n Toowoomba --...,Australia(内销),HW9,CW201912160184-AU1,深圳备货区,FSCK19122386195-WH,2019-12-27 13:31:47,74674,48813,0,光纤模块,Generic Compatible 10GBASE-ER SFP+ 1310nm 40km...,SFP-10GER-31,0.02,0.6912kg,非欧盟,Claire.Li,Australian credit card,AUD,72.60,2019-12-16,FS191215562041,AUD,1,35.0323,1.4558,72.60,72.60,NaN,20010708406,10%,6.60,6.60,66.00,66.00,公司,NaN,J8PZ50005823,DHL 2-5 Business Days,6.34263e+09,否,15.0,15.0,0.0,0.0,44.8800,0.

In [0]:
#df_all2['DeliveryMethod'].mask(df_all2['运单号'] == 'J8PZ50005857', 'Startrack', inplace=True)
df_all2.loc[25438,'DeliveryMethod'] = 'Startrack'
df_all2.loc[21498,'DeliveryMethod'] = 'Startrack'
df_all2.loc[21497,'DeliveryMethod'] = 'Startrack'
df_all2.loc[21380,'DeliveryMethod'] = 'Fedex'
df_all2.loc[15697,'DeliveryMethod'] = 'Startrack'
df_all2.loc[21608,'DeliveryMethod'] = 'Startrack'
df_all2.loc[21503,'DeliveryMethod'] = 'Startrack'
df_all2.loc[21384,'DeliveryMethod'] = 'Fedex'

In [0]:
df_all2 = df_all2[df_all2['DeliveryMethod'] != 'Fedex']

In [87]:
tempdf = df_all2[df_all2['DeliveryMethod'].isnull()]
tempdf

,序号,公司编号,客户新代码,客户名称,所属国家,收货地址,收货国家,运营模式,OrderNumber,发货仓库,出库单号,PickUpTime,商品ID,存货ID,虚拟ID,中文名称,货物名称,规格型号,产品重量,发货总重量,销售地区,业务员,到款方式,到款币种,到款金额,打款时间,发票号,销售币种,数量,前台售价,折算汇率,含税单价,含税金额,其中销售税,客户GST税号,GST税率,GST金额,GST分摊,不含税金额（原币）,PriceWithoutGST(AUD),个人/公司,DeliveryMethod,TrackingNumber,转运方式,转运单号,是否免运费,运费金额,分摊运费,保险费用,实际运费,采购单价,采购运费,采购成本原币,采购成本（AUD）,仓管ID,折人民币,国内采购成本,采购员,电源线,los,供应商,CG号,HF单号,IN编码,发票重开,订单单价,分摊保费,8月运费对账单,客户编码,含税金额.1


In [88]:
df_all2['DeliveryMethod'].unique()

array(['StarTrack Premium 1-3 Business Days', 'TNT Overnight Express',
       'TNT Road Express 1-4 Business Days', 'StarTrack Premium',
       'StarTrack Premium 1-3 Business', 'TNT 9:00 Express Service',
       'StarTrack Fixed Price Premium', 'StarTrack 1-5 Business Days',
       'TNT 9:00 Express', 'TNT 12:00 Express', 'TNT Express PC',
       'TNT SameDay', 'Startrack', 'TNT 10:00 Express'], dtype=object)

In [0]:
df_all2['DeliveryMethod2']= df_all2['DeliveryMethod']

In [0]:
#df['Desk'] = df['Desk'].map( {'N': 0, 'Y': 1, 'YY': 1} )
df_all2['DeliveryMethod2'] = df_all2['DeliveryMethod2'].replace(['StarTrack Premium 1-3 Business Days'
  , 'StarTrack Premium', 'StarTrack Premium 1-3 Business',
  'StarTrack Fixed Price Premium','StarTrack 1-5 Business Days',
  ], 'Startrack')
df_all2['DeliveryMethod2'] = df_all2['DeliveryMethod2'].replace(['TNT Overnight Express'
  , 'TNT Road Express 1-4 Business Days', 'TNT 9:00 Express Service',
  'TNT 12:00 Express','TNT Express PC','TNT SameDay', 'TNT 10:00 Express','TNT 9:00 Express'
  ], 'TNT')

In [91]:
df_all2['DeliveryMethod'].unique()

array(['StarTrack Premium 1-3 Business Days', 'TNT Overnight Express',
       'TNT Road Express 1-4 Business Days', 'StarTrack Premium',
       'StarTrack Premium 1-3 Business', 'TNT 9:00 Express Service',
       'StarTrack Fixed Price Premium', 'StarTrack 1-5 Business Days',
       'TNT 9:00 Express', 'TNT 12:00 Express', 'TNT Express PC',
       'TNT SameDay', 'Startrack', 'TNT 10:00 Express'], dtype=object)

In [92]:
df_all2['DeliveryMethod2'].value_counts()

Startrack    16611
TNT           6724
Name: DeliveryMethod2, dtype: int64

In [93]:
df_all2.sample()

,序号,公司编号,客户新代码,客户名称,所属国家,收货地址,收货国家,运营模式,OrderNumber,发货仓库,出库单号,PickUpTime,商品ID,存货ID,虚拟ID,中文名称,货物名称,规格型号,产品重量,发货总重量,销售地区,业务员,到款方式,到款币种,到款金额,打款时间,发票号,销售币种,数量,前台售价,折算汇率,含税单价,含税金额,其中销售税,客户GST税号,GST税率,GST金额,GST分摊,不含税金额（原币）,PriceWithoutGST(AUD),个人/公司,DeliveryMethod,TrackingNumber,转运方式,转运单号,是否免运费,运费金额,分摊运费,保险费用,实际运费,采购单价,采购运费,采购成本原币,采购成本（AUD）,仓管ID,折人民币,国内采购成本,采购员,电源线,los,供应商,CG号,HF单号,IN编码,发票重开,订单单价,分摊保费,8月运费对账单,客户编码,含税金额.1,DeliveryMethod2
15514,933,NaN,C201107578,Rob,Australia,"Rob Georgievski\nUnit 4, 9 Fitzpatrick Street\...",Australia(内销),HW9,CW20191015327-AU4,武汉半成品仓,FSCK19101545456-WH,2019-10-24 12:06:12,48856,48568,0,光纤模块,Dell QSFP28-100G-SR4 Compatible 100GBASE-SR4 Q...,QSFP28-SR4-100G,0.095,4.032kg,非欧盟,Molica.Li,Credit card(975),AUD,7513.66,2019-10-15,FS191014378762-P2,AUD,8,98.9147,1.4558,155.23,1241.86,NaN,NaN,10%,683.06,112.9,1128.96,1128.96,个人,StarTrack Premium,J8PZ50003698,DHL 2-5 Business Days,2.51716e+09,是,0.0,0.0,0.0,0.0,124.1856,0.0,993.4848,993.48,2944,"5,547.48",2960.0,lisa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.12,NaN,NaN,201107578.0,NaN,Startrack


In [0]:
df_all2 = df_all2[df_all2['PriceWithoutGST(AUD)'] >= 0]

In [0]:
#table = pd.pivot_table(df_all2,index=['DeliveryMethod2','OrderNumber','TrackingNumber'],aggfunc={'PriceWithoutGST(AUD)':np.sum})
#table

In [107]:
table_groupby = df_all2.groupby(['DeliveryMethod2','OrderNumber','TrackingNumber'], as_index=False)['PriceWithoutGST(AUD)'].sum()
table_groupby

,DeliveryMethod2,OrderNumber,TrackingNumber,PriceWithoutGST(AUD)
0,Startrack,AL20191010006-AU,J8PZ50003176,676.20
1,Startrack,AL20191010006-AU3,J8PZ50003521,1470.00
2,Startrack,AL20191010006-AU4,J8PZ50004049,4753.00
3,Startrack,BH20190118007-AU,5DYZ00003045,375.25
4,Startrack,BH20190130001-AU,5DYZ00003291,366.70
...,...,...,...,...
10844,TNT,ZQ20191121057-AU,428104095,1260.44
10845,TNT,ZQ20191122094-AU,429396575,1905.00
10846,TNT,ZQ20191125041-AU,430121510,312.60
10847,TNT,ZQ20191127056-AU3,FMO200114347,62372.00


In [97]:
table.columns

Index(['PriceWithoutGST(AUD)'], dtype='object')

In [98]:
import glob

path = r'/content/drive/My Drive/tnt' # use your path
all_files = glob.glob(os.path.join(path, "*.CSV"))
df_tnt = pd.concat((pd.read_csv(f) for f in all_files))
#li = []
#
#for filename in all_files:
#    df = pd.read_csv(filename, index_col=None, header=0)
#    li.append(df)

#frame = pd.concat(li, axis=0, ignore_index=True)
df_tnt.sample(3)

,Account Number,Account Name,Year,Week,Invoice Date,Invoice Number,Pick Up Date,Consignment No,Reference,Sender Company,Sender City,Sender Postcode,Receiver Company,Receiver City,Receiver Postcode,Origin,Destination,Payer,International/Domestic,Div/Prod Code,Rating Code 3,Rating Code 4,Dangerous Goods,Total Pieces,Billed Weight,Volume,Vol Wt Id,Declared Weight,Freight Amount,Automatic Warranty,Extended Warranty,Surcharge,GST,Total Amount
9,30024251,FS.COM,2020,23,09/11/19,57325777,06/11/19,419948115,FS191103323157,FS.COM,CLEELAND,3175.0,CHAPTECH ON Q COMMUNICATIONS,BRISBANE,4000.0,MEL,BNE,T,D,D 76,NaN,NaN,N,3,18,0.0704,V,10,24.76,0,0,4.43,2.92,32.11
22,30024251,FS.COM,2020,16,21/09/19,56987279,18/09/19,295413823,FS190917599999,FS.COM,CLEELAND,3175.0,FLYINGBARK PRODUCTIONS PTY,CAMPERDOWN,2050.0,MEL,SYD,T,D,D 76,NaN,NaN,N,1,19,0.0746,V,14,15.59,0,0,2.85,1.84,20.28
41,30024251,FS.COM,2020,24,16/11/19,57375449,15/11/19,425050776,FS191028489395,FS.COM,CLEELAND,3175.0,AARNET,THE LEVELS,5095.0,MEL,ADL,T,D,D 75,NaN,NaN,N,1,2,0.0045,V,1,12.01,0,0,2.45,1.45,15.91


In [0]:
df_tnt = df_tnt.rename({'Consignment No':'TrackingNumber'}, axis=1)

In [0]:
df_tnt['TrackingNumber'] = df_tnt['TrackingNumber'].astype(str)
table_groupby['TrackingNumber'] = table_groupby['TrackingNumber'].astype(str)

In [0]:
df_tnt_lookup = df_tnt.merge(
                     table_groupby,  
                     on ='TrackingNumber',  
                     how ='left') 


In [131]:
df_tnt_lookup.loc[df_tnt_lookup['TrackingNumber'] == '287817482']

,Account Number,Account Name,Year,Week,Invoice Date,Invoice Number,Pick Up Date,TrackingNumber,Reference,Sender Company,Sender City,Sender Postcode,Receiver Company,Receiver City,Receiver Postcode,Origin,Destination,Payer,International/Domestic,Div/Prod Code,Rating Code 3,Rating Code 4,Dangerous Goods,Total Pieces,Billed Weight,Volume,Vol Wt Id,Declared Weight,Freight Amount,Automatic Warranty,Extended Warranty,Surcharge,GST,Total Amount,DeliveryMethod2,OrderNumber,PriceWithoutGST(AUD)
1067,30024251,FS.COM,2020,14,07/09/19,56895087,04/09/19,287817482,FS190903619562,FS.COM,CLEELAND,3175.0,SPEEDCAST,BOTANY,2019.0,MEL,SYD,T,D,D 75,NaN,NaN,N,1,2,0.0045,V,1,12.01,0,0,2.5,1.45,15.96,TNT,CW20190903234-AU1,53.0


In [101]:
df_tnt_lookup.sample(3)

,Account Number,Account Name,Year,Week,Invoice Date,Invoice Number,Pick Up Date,TrackingNumber,Reference,Sender Company,Sender City,Sender Postcode,Receiver Company,Receiver City,Receiver Postcode,Origin,Destination,Payer,International/Domestic,Div/Prod Code,Rating Code 3,Rating Code 4,Dangerous Goods,Total Pieces,Billed Weight,Volume,Vol Wt Id,Declared Weight,Freight Amount,Automatic Warranty,Extended Warranty,Surcharge,GST,Total Amount,序号,公司编号,客户新代码,客户名称,所属国家,收货地址,...,折算汇率,含税单价,含税金额,其中销售税,客户GST税号,GST税率,GST金额,GST分摊,不含税金额（原币）,PriceWithoutGST(AUD),个人/公司,DeliveryMethod,转运方式,转运单号,是否免运费,运费金额,分摊运费,保险费用,实际运费,采购单价,采购运费,采购成本原币,采购成本（AUD）,仓管ID,折人民币,国内采购成本,采购员,电源线,los,供应商,CG号,HF单号,IN编码,发票重开,订单单价,分摊保费,8月运费对账单,客户编码,含税金额.1,DeliveryMethod2
524,30024251,FS.COM,2020,36,08/02/20,57904970,04/02/20,FMO200204881,202001310130-P2,FS.COM,DANDENONG SOUTH,3175.0,ACHIN JAIN,FORTITUDE VALLEY,4006.0,MEL,BNE,S,D,D 76,NaN,NaN,N,1,15,0.0581,V,10,24.76,0,0,4.56,2.93,32.25,127.0,G341010565,651181811,Telstra Corp,Australia,Achin Jain\nNextDC Brisbane B2 site\n Fortitud...,...,1.4558,78.10,781.00,NaN,NaN,10%,252.20,71.00,710.0,710.0,个人,TNT Road Express 1-4 Business Days,NaN,NaN,是,0.0,0.0,0.0,0.0,178.000,0.0,1780.000,370.03,4253.0,"3,415.44",1780.0,Iris.Liu,NaN,NaN,112X,CG1911205487-AU,NaN,NaN,NaN,71.0,NaN,NaN,NaN,NaN,TNT
1036,30024251,FS.COM,2020,33,18/01/20,57764694,16/01/20,FMO200116642,191230308691-P2,FS.COM,DANDENONG SOUTH,3175.0,MITCHELL DALLEY,OAKVILLE,2765.0,MEL,SYD,S,D,D 76,NaN,NaN,N,1,8,0.0290,V,4,20.59,0,0,2.85,2.34,25.78,34.0,NaN,B301139181,Mitchell Dalley,Australia,Mitchell Dalley\n558 Old Pitt Town Rd\n Oakvil...,...,1.4558,3.93,3.93,NaN,NaN,10%,6.23,0.33,3.6,3.6,个人,TNT Road Express 1-4 Business Days,DHL 2-5 Business Days,6.30055e+09,否,0.0,0.0,0.0,0.0,3.168,0.0,3.168,3.17,3290.0,17.32,2.0,Sydnie.Wang,NaN,NaN,440X,PZQ20190925031-01,NaN,NaN,NaN,3.6,NaN,NaN,NaN,NaN,TNT
1743,30024251,FS.COM,2020,19,12/10/19,57129250,11/10/19,407303470,FS191002470653,FS.COM,CLEELAND,3175.0,RUSSELL MINERAL EQUIPMENT,TOOWOOMBA,4350.0,MEL,TBA,T,D,D 75,NaN,NaN,N,1,2,0.0045,V,1,21.62,0,0,4.39,2.60,28.61,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
tempdf = df_tnt_lookup[df_tnt_lookup['PriceWithoutGST(AUD)'].isnull()]
tempdf

,Account Number,Account Name,Year,Week,Invoice Date,Invoice Number,Pick Up Date,TrackingNumber,Reference,Sender Company,Sender City,Sender Postcode,Receiver Company,Receiver City,Receiver Postcode,Origin,Destination,Payer,International/Domestic,Div/Prod Code,Rating Code 3,Rating Code 4,Dangerous Goods,Total Pieces,Billed Weight,Volume,Vol Wt Id,Declared Weight,Freight Amount,Automatic Warranty,Extended Warranty,Surcharge,GST,Total Amount,DeliveryMethod2,OrderNumber,PriceWithoutGST(AUD)
14,30024251,FS.COM,2020,39,29/02/20,58038793,25/02/20,FMO200225076,FS200223454207,FS.COM,DANDENONG SOUTH,3175.0,GAVIN BULLEN,ALBERT PARK BARRACK,3004.0,MEL,MEL,S,D,D 75,NaN,NaN,N,1,2,0.0055,V,1,9.61,0,0,2.01,1.16,12.78,NaN,NaN,NaN
57,30024251,FS.COM,2020,38,22/02/20,57988901,17/02/20,FMO200217552,FS200216759301,FS.COM,DANDENONG SOUTH,3175.0,RADEK TKACZYK,BRISBANE CITY,4000.0,MEL,BNE,S,D,D 76,NaN,NaN,N,1,9,0.0360,V,7,24.76,0,0,4.56,2.93,32.25,NaN,NaN,NaN
64,30024251,FS.COM,2020,38,22/02/20,57988901,18/02/20,983509835793,NaN,NaN,MARLESTON,5033.0,FS COM,DANDENONG SOUTH,3175.0,ADL,MEL,S,D,D 76,NaN,NaN,N,1,4,0.0160,V,3,35.59,0,0,2.87,3.85,42.31,NaN,NaN,NaN
74,30024251,FS.COM,2020,38,22/02/20,57988901,19/02/20,151125311,FS202002170041-P2,FS.COM,BANGHOLME,3175.0,MEGAPORT (AUSTRALIA) PTY LTD,FORTITUDE VALLEY,4006.0,MEL,BNE,S,D,D 76,NaN,NaN,N,5,53,0.2087,V,40,46.61,0,0,8.58,5.52,60.71,NaN,NaN,NaN
111,30024251,FS.COM,2020,38,22/02/20,57988901,22/02/20,57954052,Debit Adjustment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0000,NaN,0,5.00,0,0,0.00,0.50,5.50,NaN,NaN,NaN
112,30024251,FS.COM,2020,38,22/02/20,57988901,22/02/20,57954938,Debit Adjustment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0000,NaN,0,20.00,0,0,0.00,2.00,22.00,NaN,NaN,NaN
137,30024251,FS.COM,2020,37,15/02/20,57949496,12/02/20,146535755,NaN,FS.COM,BANGHOLME,3175.0,SYNDETICOM,BALGOWLAH,2093.0,MEL,SYD,S,D,D 75,NaN,NaN,N,1,6,0.0216,V,3,19.22,0,0,4.02,2.32,25.56,NaN,NaN,NaN
229,30024251,FS.COM,2020,35,01/02/20,57856354,29/01/20,LET000001939,FMO200114796,Mitchell Family Grocer (tradin,MITCHELL,2911.0,FS.COM,DANDENONG SOUTH,3175.0,CBA,MEL,R,D,D 76,NaN,NaN,N,1,1,0.0038,W,1,35.59,0,0,2.87,3.85,42.31,NaN,NaN,NaN
276,30024251,FS.COM,2020,34,25/01/20,57809052,22/01/20,209630890,FMO200117523 PER ENQUIRY,F O C - ENFIELD ACCOUNT,ENFIELD,2136.0,FS.COM,DANDENONG,3175.0,SYD,MEL,R,D,D 76,NaN,NaN,N,1,1,0.0030,W,1,15.59,0,0,2.85,1.84,20.28,NaN,NaN,NaN
360,30024251,FS.COM,2020,33,18/01/20,57764694,17/01/20,FMO200117523,FS200115238236,FS.COM,DANDENONG SOUTH,3175.0,STEVEN YANG,CHATSWOOD,2067.0,MEL,SYD,S,D,D 75,NaN,NaN,N,1,1,0.0035,W,1,32.01,0,0,2.50,3.45,37.96,NaN,NaN,NaN
